### Restart and Run All Cells

In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2021
quarter = 4

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

In [3]:
format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [4]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2021 AND quarter <= 4) 
OR (year = 2021-1 AND quarter >= 4+1)
ORDER BY year DESC, quarter DESC



In [5]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.head()

,name,year,quarter,q_amt,Counter
3,ADVANC,8084,10,26922146,4
17,AOT,8084,10,-16322014,4
24,ASP,8084,10,978355,4
31,BAY,8084,10,33794188,4
32,BBL,8084,10,26507039,4


In [6]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2021 AND quarter <= 4-1) 
OR (year = 2021-1 AND quarter >= 4) 
ORDER BY year DESC, quarter DESC


In [7]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,2S,8083,10,"788,297",4
1,AAV,8083,10,"-6,769,190",4
2,ACE,8083,10,"1,392,091",4
3,ADVANC,8083,10,"27,223,149",4
4,AEONTS,8083,10,"3,826,113",4


In [8]:
dfp.name.unique().shape

(338,)

In [9]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(339, 15)

In [10]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(403, 9)

In [11]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ADVANC,2021,Q4,"26,922,146","27,223,149","-301,003",-1.11%
1,AOT,2021,Q4,"-16,322,014","-14,892,318","-1,429,696",-9.60%
2,ASP,2021,Q4,"978,355","907,856","70,499",7.77%
3,BAY,2021,Q4,"33,794,188","30,793,995","3,000,193",9.74%
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%


In [12]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2021 AND quarter = 'Q4'



24

In [13]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.head()

,name,id
0,2S,727
1,A,1
2,AAV,732
3,ACE,698
4,ADVANC,6


In [14]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,ADVANC,2021,Q4,"26,922,146","27,223,149","-301,003",-1.11%,6
1,AOT,2021,Q4,"-16,322,014","-14,892,318","-1,429,696",-9.60%,24
2,ASP,2021,Q4,"978,355","907,856","70,499",7.77%,40
3,BAY,2021,Q4,"33,794,188","30,793,995","3,000,193",9.74%,49
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%,50


In [15]:
rcds = df_ins.values.tolist()
len(rcds)

27

In [16]:
#for rcd in rcds:
#    print(rcd)

In [17]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [18]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [19]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ADVANC,2021,Q4,"26,922,146","27,223,149","-301,003",-1.11%
2,ASP,2021,Q4,"978,355","907,856","70,499",7.77%
3,BAY,2021,Q4,"33,794,188","30,793,995","3,000,193",9.74%
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%
5,BFIT,2021,Q4,"906,775","924,632","-17,857",-1.93%


In [20]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ADVANC,2021,Q4,"26,922,146","27,223,149","-301,003",-1.11%
2,ASP,2021,Q4,"978,355","907,856","70,499",7.77%
3,BAY,2021,Q4,"33,794,188","30,793,995","3,000,193",9.74%
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%
5,BFIT,2021,Q4,"906,775","924,632","-17,857",-1.93%


In [21]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%
6,CIMBT,2021,Q4,"2,440,554","1,531,200","909,354",59.39%
13,KCE,2021,Q4,"2,426,284","2,105,875","320,409",15.22%
14,KKP,2021,Q4,"6,318,052","5,402,784","915,268",16.94%
21,PTTEP,2021,Q4,"38,863,595","30,745,139","8,118,456",26.41%


In [22]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%
6,CIMBT,2021,Q4,"2,440,554","1,531,200","909,354",59.39%
13,KCE,2021,Q4,"2,426,284","2,105,875","320,409",15.22%
14,KKP,2021,Q4,"6,318,052","5,402,784","915,268",16.94%
21,PTTEP,2021,Q4,"38,863,595","30,745,139","8,118,456",26.41%


In [23]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
6,CIMBT,2021,Q4,"2,440,554","1,531,200","909,354",59.39%,104
21,PTTEP,2021,Q4,"38,863,595","30,745,139","8,118,456",26.41%,384
26,TTB,2021,Q4,"10,474,045","8,909,595","1,564,450",17.56%,541
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%,50
14,KKP,2021,Q4,"6,318,052","5,402,784","915,268",16.94%,255


In [24]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%,50
6,CIMBT,2021,Q4,"2,440,554","1,531,200","909,354",59.39%,104
13,KCE,2021,Q4,"2,426,284","2,105,875","320,409",15.22%,249
14,KKP,2021,Q4,"6,318,052","5,402,784","915,268",16.94%,255
21,PTTEP,2021,Q4,"38,863,595","30,745,139","8,118,456",26.41%,384


In [25]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
4,BBL,2021,Q4,"26,507,039","22,586,660","3,920,379",17.36%,50
6,CIMBT,2021,Q4,"2,440,554","1,531,200","909,354",59.39%,104
13,KCE,2021,Q4,"2,426,284","2,105,875","320,409",15.22%,249
14,KKP,2021,Q4,"6,318,052","5,402,784","915,268",16.94%,255
21,PTTEP,2021,Q4,"38,863,595","30,745,139","8,118,456",26.41%,384


In [26]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
0,BCT,1
1,INSET,1
2,NRF,1
3,OR,1
4,UOBKH,1
